In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from collections import namedtuple

import cologne_phonetics
import jellyfish
import joblib
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from metaphone import doublemetaphone
from mpire import WorkerPool
import numpy as np
from pyphonetics import RefinedSoundex
from sklearn.model_selection import train_test_split
from spellwise import CaverphoneOne, CaverphoneTwo
import pandas as pd
import torch
from tqdm import tqdm
import wandb

from src.data.filesystem import fopen
from src.data.normalize import normalize_freq_names
from src.data.utils import load_dataset, select_frequent_k
from src.eval import metrics
from src.models.utils import remove_padding
from src.models.cluster import read_clusters, get_validation_results
from src.models.swivel import SwivelModel

In [ ]:
# config

given_surname = "surname"
vocab_size = 610000 if given_surname == "given" else 2100000
eval_size = 200000
sample_size = 1000
embed_dim = 100
NAMA_MAX_CLUSTERS = 20
n_jobs = 1
verbose = True
num_matches = 1000  # Number of candidates to consider

Config = namedtuple("Config", [
    "eval_path",
    "test_path",
    "freq_path",
    "embed_dim",
    "swivel_vocab_path",
    "swivel_model_path",
    "tfidf_path",
    "ensemble_model_path",
    "cluster_path",
    "partition_path",
])
config = Config(
    eval_path=f"s3://familysearch-names/processed/tree-hr-{given_surname}-train.csv.gz",
    test_path=f"s3://familysearch-names/processed/tree-hr-{given_surname}-test.csv.gz",
    freq_path=f"s3://familysearch-names/processed/tree-preferred-{given_surname}-aggr.csv.gz",
    embed_dim=embed_dim,
    swivel_vocab_path=f"s3://nama-data/data/models/fs-{given_surname}-swivel-vocab-{vocab_size}-augmented.csv",
    swivel_model_path=f"s3://nama-data/data/models/fs-{given_surname}-swivel-model-{vocab_size}-{embed_dim}-augmented.pth",
    tfidf_path=f"s3://nama-data/data/models/fs-{given_surname}-tfidf.joblib",
    ensemble_model_path=f"s3://nama-data/data/models/fs-{given_surname}-ensemble-model-{vocab_size}-{embed_dim}-augmented-100.joblib",    
    cluster_path=f"s3://nama-data/data/models/fs-{given_surname}-cluster-names.csv",
    partition_path=f"s3://nama-data/data/models/fs-{given_surname}-cluster-partitions.csv",
)

In [ ]:
np.set_printoptions(suppress=True)
tqdm.pandas()

wandb.init(
    project="nama",
    entity="nama",
    name="90_compare_coders",
    group=given_surname,
    notes="test",
    config=config._asdict(),
)

### Load data

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
input_names_eval, weighted_actual_names_eval, candidate_names_eval = load_dataset(config.eval_path, is_eval=True)
input_names_test, weighted_actual_names_test, candidate_names_test = load_dataset(config.test_path, is_eval=True)

In [ ]:
# make all candidate_names_eval are in actual_names_eval
actual_names_eval = set([name for wans in weighted_actual_names_eval for name, _, _ in wans])
candidate_names_eval = np.array(list(actual_names_eval))
del actual_names_eval
print(len(candidate_names_eval))

In [ ]:
# get the most-frequent eval_size names
input_names_eval, weighted_actual_names_eval, candidate_names_eval = \
    select_frequent_k(input_names_eval,
                      weighted_actual_names_eval,
                      candidate_names_eval,
                      eval_size)

In [ ]:
print("input_names_eval", len(input_names_eval))
print("candidate_names_eval", len(candidate_names_eval))

print("input_names_test", len(input_names_test))
print("candidate_names_test", len(candidate_names_test))

In [ ]:
freq_df = pd.read_csv(config.freq_path, na_filter=False)
name_freq = normalize_freq_names(freq_df, is_surname=given_surname != "given", add_padding=True)
freq_df = None

In [ ]:
vocab_df = pd.read_csv(fopen(config.swivel_vocab_path, "rb"))
swivel_vocab = {name: _id for name, _id in zip(vocab_df["name"], vocab_df["index"])}

In [ ]:
swivel_model = SwivelModel(len(swivel_vocab), config.embed_dim)
swivel_model.load_state_dict(torch.load(fopen(config.swivel_model_path, "rb"), map_location=torch.device(device)))
swivel_model.to(device)
swivel_model.eval()

In [ ]:
tfidf_vectorizer = joblib.load(fopen(config.tfidf_path, mode='rb'))

In [ ]:
ensemble_model = joblib.load(fopen(config.ensemble_model_path, mode='rb'))

In [ ]:
name_cluster = read_clusters(config.cluster_path)
print("name_cluster", len(name_cluster))

In [ ]:
for ix, (key, value) in enumerate(name_cluster.items()):
    print(key, value)
    if ix > 20:
        break

In [ ]:
if given_surname == "given":
    cluster_partition = None
else:
    cluster_partition_df = pd.read_csv(config.partition_path)
    cluster_partition = {cluster: (start_partition, n_partitions) \
                      for cluster, start_partition, n_partitions in \
                         zip(cluster_partition_df["cluster"], 
                             cluster_partition_df["start_partition"],
                             cluster_partition_df["n_partitions"])}

In [ ]:
input_names_test, weighted_actual_names_test, candidate_names_test = load_dataset(config.test_path, is_eval=True)

In [ ]:
# remove oov names from input_names_test and remove in-vocab names from candidate_names_test
# so we only compare in-vocab against out-of-vocab
input_names_test_iv = []
weighted_actual_names_test_iv = []
candidate_names_test_oov = set()
for input_name, wans in zip(input_names_test, weighted_actual_names_test):
    if input_name not in swivel_vocab:
        continue
    wans_oov = []
    sum_freq = 0
    for name, weight, freq in wans:
        if name in swivel_vocab or freq == 0:
            continue
        wans_oov.append((name, freq))
        sum_freq += freq
    wans_oov = [(name, freq / sum_freq, freq) for name, freq in wans_oov]
    if len(wans_oov) == 0:
        continue
    input_names_test_iv.append(input_name)
    weighted_actual_names_test_iv.append(wans_oov)
    for name, _, _ in wans_oov:
        candidate_names_test_oov.add(name)
candidate_names_test_oov = list(candidate_names_test_oov)
# for candidate_name in candidate_names_test:
#     if candidate_name in swivel_vocab:
#         continue
#     candidate_names_test_oov.append(candidate_name)

In [ ]:
print(len(input_names_test))
print(len(input_names_test_iv))
print(len(candidate_names_test))
print(len(candidate_names_test_oov))

In [ ]:
print(input_names_test_iv[0:5])
print(weighted_actual_names_test_iv[0:5])
print(candidate_names_test_oov[0:5])

In [ ]:
input_names_test = input_names_test_iv
weighted_actual_names_test = weighted_actual_names_test_iv
candidate_names_test = np.array(candidate_names_test_oov)

In [ ]:
print(num_matches)

### Other Models

In [ ]:
# various coders
caverphone_one = CaverphoneOne()
caverphone_two = CaverphoneTwo()
refined_soundex = RefinedSoundex()

In [ ]:
coding_algos = [
    "nama",
    "nama-05",
    "nama-50",
    "nama-80",
    "soundex",
    "nysiis",
    "metaphone",
    "caverphone1",
    "caverphone2",
    "refined_soundex",
    #     "double_metaphone",  # bad implementation?
    "cologne_phonetics",
]

In [ ]:
# test double metaphone
name = "smith"
cand_name = "schmidt"
dm1 = doublemetaphone(name)
dm2 = doublemetaphone(cand_name)
similarity = 1.0 if any(code in dm2 for code in dm1) else 0.0
print("dm1", dm1)
print("dm2", dm2)
print("similarity", similarity)

### Similarity functions

In [ ]:
def get_codes(name, algo):
    name = remove_padding(name)
    if algo == "caverphone1":
        return [caverphone_one._pre_process(name)]
    elif algo == "caverphone2":
        return [caverphone_two._pre_process(name)]
    elif algo == "refined_soundex":
        return [refined_soundex.phonetics(name)]
    elif algo == "double_metaphone":
        return doublemetaphone(name)
    elif algo == "cologne_phonetics":
        return [cologne_phonetics.encode(name)[0][1]]
    elif algo == "soundex":
        return [jellyfish.soundex(name)]
    elif algo == "nysiis":
        return [jellyfish.nysiis(name)]
    elif algo == "metaphone":
        return [jellyfish.metaphone(name)]

In [ ]:
def calc_similarity_to(name, name2codes):
    codes1 = set(name2codes[name])

    def calc_similarity(row):
        cand_name = row[0]
        code2 = name2codes[cand_name][0]  # code2 is the code cand_name is indexed under
        return 1.0 if code2 in codes1 else 0.0

    return calc_similarity

In [ ]:
def get_similars(shared, name=""):
    candidate_names, k, name2codes = shared
    scores = np.apply_along_axis(calc_similarity_to(name, name2codes), 1, candidate_names[:, None])
    sorted_scores_idx = np.argsort(scores)[::-1][:k]
    candidate_names = candidate_names[sorted_scores_idx]
    candidate_scores = scores[sorted_scores_idx]

    return list(zip(candidate_names, candidate_scores))

# Evaluate each algorithm

In [ ]:
def evaluate_algos(coding_algos,
                   swivel_vocab,
                   swivel_model,
                   name_freq,
                   name_cluster,
                   cluster_partition,
                   tfidf_vectorizer,
                   ensemble_model,
                   input_names,
                   weighted_actual_names,
                   candidate_names):

    figure, ax = plt.subplots(1, 1, figsize=(20, 15))
    ax.set_title("PR at threshold")
    colors = cm.rainbow(np.linspace(0, 1, len(coding_algos)))
    all_names = list(set(input_names).union(set(candidate_names)))

    for algo, color in zip(coding_algos, colors):
        print(algo)
        if algo.startswith("nama"):
            if algo == "nama":
                search_threshold = 0
                max_clusters = 1  # return just one cluster
            else:
                _, search_threshold = algo.split('-')
                search_threshold = int(search_threshold) / 100.0
                max_clusters = NAMA_MAX_CLUSTERS
            results = get_validation_results(
                input_names_eval=input_names,
                weighted_actual_names_eval=weighted_actual_names,
                candidate_names_eval=candidate_names,
                name_freq=name_freq,
                name_cluster=name_cluster,
                cluster_partition=cluster_partition,
                swivel_model=swivel_model,
                swivel_vocab=swivel_vocab,
                tfidf_vectorizer=tfidf_vectorizer,
                ensemble_model=ensemble_model,
                search_threshold=search_threshold,
                num_matches=num_matches,
                max_clusters=max_clusters,
                sample_size=sample_size,
                validation_sizes=[0],
                n_jobs=n_jobs,
                verbose=verbose) 
            precision = results['precisions'][0][search_threshold]
            recall = results['recalls'][0][search_threshold]
            avg_partitions = results['avg_partitions'][0][search_threshold]
        else:
            name2codes = {name: get_codes(name, algo) for name in all_names}
            with WorkerPool(shared_objects=(candidate_names, num_matches, name2codes)) as pool:
                similar_names_scores = pool.map(get_similars, input_names, progress_bar=True)
            similar_names = [[name for name, _ in name_similarities] for name_similarities in similar_names_scores]
            names = np.array(list(list(cell[0] for cell in row) for row in similar_names_scores), dtype="O")
            scores = np.array(list(list(cell[1] for cell in row) for row in similar_names_scores), dtype="f8")
            total = max(scores.sum(axis=1))
            print("max sum of scores", total)
            if total == num_matches:
                print("WARNING: need to increase num_matches")
            similar_names_scores = np.dstack((names, scores))
            precision = metrics.avg_precision_at_threshold(weighted_actual_names, similar_names_scores, 0.5)
            recall = metrics.avg_weighted_recall_at_threshold(weighted_actual_names, similar_names_scores, 0.5)
            avg_partitions = 1.0
        print(f"precision={precision} recall={recall} avg_partitions={avg_partitions}")
        precisions = [precision]
        recalls = [recall]
        ax.plot(recalls, precisions, "o--", color=color, label=algo)

    ax.legend()
    plt.xlim([0, 1.0])
    plt.ylim([0, 1.0])
    plt.show()

#### on in-vocabulary names (training data)

In [ ]:
_, input_names_sample, _, weighted_actual_names_sample = \
    train_test_split(input_names_eval, weighted_actual_names_eval, test_size=sample_size)
candidate_names_sample = candidate_names_eval

print("input_names", len(input_names_sample))
print("weighted_actual_names", len(weighted_actual_names_sample))
print("candidate_names", len(candidate_names_sample))

In [ ]:
%%time
evaluate_algos(coding_algos,
                   swivel_vocab,
                   swivel_model,
                   name_freq,
                   name_cluster,
                   cluster_partition,
                   tfidf_vectorizer,
                   ensemble_model,
                   input_names_sample,
                   weighted_actual_names_sample,
                   candidate_names_sample)

#### on out-of-vocabulary names (test data)

In [ ]:
_, input_names_sample, _, weighted_actual_names_sample = \
    train_test_split(input_names_test, weighted_actual_names_test, test_size=sample_size)
candidate_names_sample = candidate_names_test

print("input_names", len(input_names_sample))
print("weighted_actual_names", len(weighted_actual_names_sample))
print("candidate_names", len(candidate_names_sample))
print("all names", len(set(input_names_sample).union(set(candidate_names_sample))))

In [ ]:
n_zero = n_one = n_two = 0
for input_name, wans in zip(input_names_sample, weighted_actual_names_sample):
    for actual_name, _, _ in wans:
        if input_name in swivel_vocab and actual_name in swivel_vocab:
            n_two += 1
        elif input_name in swivel_vocab or actual_name in swivel_vocab:
            n_one += 1
        else:
            n_zero += 1
print("two names in vocab (should not be possible)", n_two)
print("one name in vocab", n_one)
print("zero names in vocab", n_zero)

In [ ]:
%%time
evaluate_algos(coding_algos,
                   swivel_vocab,
                   swivel_model,
                   name_freq,
                   name_cluster,
                   cluster_partition,
                   tfidf_vectorizer,
                   ensemble_model,
                   input_names_sample,
                   weighted_actual_names_sample,
                   candidate_names_sample)

In [ ]:
wandb.finish()